In [22]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Changed current working directory to: /Users/falcon/Developments/trading
Initialized project with base directory: /Users/falcon/Developments/trading


In [23]:
import models.market as market
import importlib
importlib.reload(market)

market = market.Market.from_json_file()

In [24]:
df = market.to_df()
df.head()

,maxLeverage,name,onlyIsolated,szDecimals,dayNtlVlm,funding,impactPxs,markPx,midPx,openInterest,oraclePx,premium,prevDayPx
0,50,BTC,False,5,3.360506e+08,0.000110,"[68599.5, 68601.0]",68600.0000,68600.50000,8.737719e+02,68501.0000,0.001438,69468.0000
1,50,ETH,False,4,3.319257e+08,0.000013,"[3844.55, 3844.7]",3844.6000,3844.65000,1.570506e+04,3842.9000,0.000429,3942.1000
2,20,ATOM,False,2,6.382626e+06,0.000108,"[13.284, 13.288]",13.2890,13.28600,7.952930e+04,13.2700,0.001055,13.3790
3,20,MATIC,False,1,1.225220e+07,0.000035,"[1.1898, 1.1899]",1.1899,1.18985,1.929546e+06,1.1888,0.000841,1.1829
4,20,DYDX,False,1,3.311390e+06,0.000025,"[3.94351, 3.94512]",3.9440,3.94435,3.483366e+05,3.9375,0.001526,4.1262


In [25]:
top_60_coins_df = df.sort_values(by='dayNtlVlm', ascending=False).head(60)
top_60_coins_df = top_60_coins_df.reset_index(drop=True)
top_60_coins_df.head()

,maxLeverage,name,onlyIsolated,szDecimals,dayNtlVlm,funding,impactPxs,markPx,midPx,openInterest,oraclePx,premium,prevDayPx
0,50,BTC,False,5,3.360506e+08,0.000110,"[68599.5, 68601.0]",68600.0000,68600.5000,8.737719e+02,68501.0000,0.001438,69468.0000
1,50,ETH,False,4,3.319257e+08,0.000013,"[3844.55, 3844.7]",3844.6000,3844.6500,1.570506e+04,3842.9000,0.000429,3942.1000
2,20,SOL,False,2,8.026811e+07,0.000097,"[140.32, 140.33]",140.3300,140.3250,1.668596e+05,140.1300,0.001356,147.9600
3,20,WLD,False,1,3.646147e+07,0.000093,"[9.8861, 9.8877]",9.8907,9.8869,2.661028e+05,9.8752,0.001104,10.4090
4,5,WIF,False,0,3.056737e+07,0.000144,"[2.0639, 2.064613]",2.0644,2.0642,4.232706e+06,2.0606,0.001601,2.2128


In [26]:
top_60_coins = top_60_coins_df.get('name')
first_coin = top_60_coins[0]
first_coin

'BTC'

In [27]:
from hyperliquid.info import Info
from hyperliquid.utils import constants as hl_constants
import utils.now_timestamp as now_timestamp 
import models.candles as candles
import importlib
importlib.reload(candles)
importlib.reload(now_timestamp)

info = Info(hl_constants.MAINNET_API_URL, skip_ws=True)

In [28]:
import time
start = now_timestamp.get_timestamp_from_today_midnight_utc(days=-30)
end = now_timestamp.get_timestamp_from_today_midnight_utc(days=0)
error_coins = []
for coin in top_60_coins:
    try:
        time.sleep(1)
        print(coin)
        result = info.candles_snapshot(coin, '15m', start, end)
        _candles = candles.CandleList.from_response(result)
        # _candles = candles.CandleList.save_fetch(info, coin, '15m', start, end)
        _candles.to_csv()
    except Exception as e:
        print(f"coin {coin}: not fetched")
        print(e)
        error_coins.append(coin)
        continue

BTC
ETH
SOL
WLD
WIF
DOGE
kPEPE
BNB
AVAX
NEAR
PIXEL
GALA
TIA
LINK
NFTI
ARB
FIL
STX
MATIC
RNDR
JUP
ORDI
INJ
OP
SEI
SUI
MKR
RUNE
DYM
FET
kSHIB
FTM
APT
kBONK
MINA
ALT
BCH
ATOM
STRK
PYTH
YGG
DOT
UNI
XRP
MEME
ADA
PEOPLE
ENS
AI
GMT
MANTA
XAI
DYDX
TAO
BLUR
MAVIA
AAVE
LTC
CAKE
LDO


In [14]:
# left_coins = ["DOGE", "LINK", "SEI", "BCH", "STRK", "YGG", "DOT", "UNI", "XRP", "LTC"]
# _coin = left_coins[0]
# display(start)
# display(end)
# result = info.candles_snapshot(
#     coin := _coin,
#     interval="15m",
#     startTime=start,
#     endTime=end,
# )
# result
# _candles = candles.CandleList.from_response(result)
# _df = _candles.to_df()
# _df
# _candles.to_csv()

1707523200000

1710115200000

ServerError: (500, 'null')